In [1]:
%pip install -r ../../requirements.txt

  Using cached BindsNET-0.2.9-py3-none-any.whl
  Using cached gym-0.18.0-py3-none-any.whl


You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.14/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import argparse
import torch
from bindsnet.network import Network
from bindsnet.network.nodes import Input, LIFNodes
from bindsnet.network.topology import Connection
from bindsnet.network.monitors import Monitor
from bindsnet.network.nodes import AbstractInput
from bindsnet.learning import PostPre, WeightDependentPostPre, Hebbian, MSTDP, MSTDPET

import matplotlib.pyplot as plt
import gym
from bindsnet.environment import GymEnvironment
from typing import Tuple
from collections import deque
from statistics import mean
from torch import Tensor
from bindsnet.analysis.plotting import plot_spikes
from collections import namedtuple

In [3]:
Args = namedtuple('Args', ['gamma', 'env', 'n_episode', 'hidden_dim', 'update_rule'])
FLAGS = Args(gamma=0.99, env='BreakoutDeterministic-v4', n_episode=1000, update_rule='MSTDP', hidden_dim=12)

In [4]:
rules = {
    "PostPre": PostPre,
    "WeightDependentPostPre": WeightDependentPostPre,
    "Hebbian": Hebbian,
    "MSTDP": MSTDP,
    "MSTDPET": MSTDPET,
}
s_im, s_ax = None, None

In [5]:
class SQN(object):
    def __init__(self, input_dim: int, shape: [int], output_dim: int, hidden_dim: int) -> None:
        """SQN Network
        """
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.network = Network(dt=1.0)
        self.learning_rule = rules.get(FLAGS.update_rule)
        self.time = int(self.network.dt)

        # To solve tensor formatting issues
        if FLAGS.update_rule == 'MSTDP' or FLAGS.update_rule == "HMSTDP":
            self.input = Input(n=input_dim, traces=True)
        else:
            self.input = Input(n=input_dim, shape=shape, traces=True)

        # Heterogeneous LIFNodes
        self.hidden = LIFNodes(n=hidden_dim, traces=True)
        self.output = LIFNodes(n=output_dim, traces=True)

        # First connection
        self.connection_input_hidden = Connection(
            source=self.input,
            target=self.hidden,
            update_rule=self.learning_rule,
            wmin=0,
            wmax=1,
            nu=FLAGS.gamma
        )

        # Hidden recurrent connection
        self.connection_hidden_hidden = Connection(
            source=self.hidden,
            target=self.hidden,
            update_rule=self.learning_rule,
            wmin=0,
            wmax=1,
            nu=FLAGS.gamma
        )

        # Hidden layer to Output
        self.connection_hidden_output = Connection(
            source=self.hidden,
            target=self.output,
            update_rule=self.learning_rule,
            wmin=0,
            wmax=1,
            nu=FLAGS.gamma
        )

        self.network.add_layer(
            layer=self.input, name="Input"
        )
        self.network.add_layer(
            layer=self.hidden, name="Hidden"
        )
        self.network.add_layer(
            layer=self.output, name="Output"
        )

        self.network.add_connection(
            connection=self.connection_input_hidden,
            source="Input",
            target="Hidden"
        )
        self.network.add_connection(
            connection=self.connection_hidden_hidden,
            source="Hidden",
            target="Hidden"
        )
        self.network.add_connection(
            connection=self.connection_hidden_output,
            source="Hidden",
            target="Output"
        )

        self.inputs = [
            name
            for name, layer in self.network.layers.items()
            if isinstance(layer, AbstractInput)
        ]

        # To record outputs
        self.network.add_monitor(
            Monitor(self.output, ["s"], time=self.time),
            name="output_monitor"
        )
        self.network.add_monitor(
            Monitor(self.hidden, ["s"], time=self.time),
            name="hidden_monitor"
        )
        self.network.add_monitor(
            Monitor(self.input, ["s"], time=self.time),
            name="input_monitor"
        )

        self.spike_record = {
            "Output": torch.zeros((self.time, output_dim)).to(
                self.device
            )
        }

    def run(self, inputs: dict[str, torch.Tensor], reward: [float, torch.Tensor], **kwargs) -> None:
        self.network.train(mode=True)
        return self.network.run(inputs=inputs, time=self.time, reward=reward, **kwargs)

In [6]:
class Agent(object):
    def __init__(self, input_dim: int, shape: [int], output_dim: int, hidden_dim: int) -> None:
        """Agent class that chooses an action and trains
        """
        self.sqn = SQN(input_dim, shape, output_dim, hidden_dim)
        self.input_dim = input_dim
        self.output_dim = output_dim

    def get_action(self) -> int:
        """Returns an action index
        """
        scores = self.get_Q()
        probabilities = torch.softmax(scores, dim=0)
        return torch.multinomial(probabilities, num_samples=1).item()

        _, argmax = torch.max(torch.flatten(scores), dim=0)
        # print(torch.flatten(scores), argmax.item())

        # if np.random.rand() < 0.2:
        #     return np.random.choice(self.output_dim)
        # else:
        return argmax.item()

    def get_Q(self) -> Tensor:
        """Returns `Q-value` based on output layer's spikes
        """
        return torch.sum(self.sqn.spike_record["Output"], dim=0)

In [11]:
def play_episode(env: GymEnvironment, agent: Agent) -> int:
    """Play an epsiode and train
    Args:
        env (gym.Env): gym environment (CartPole-v0)
        agent (Agent): agent will train and get action
    Returns:
        int: reward
    """
    env.reset()
    agent.sqn.network.reset_state_variables()

    done = False
    total_reward = 0

    while not done:
        env.render()
        # Select an action
        a = agent.get_action()
        # Update the state according to action a
        s, r, done, info = env.step(a)

        r = clip_reward(r)

        # Tensor shape configuration
        if FLAGS.update_rule == 'MSTDP' or FLAGS.update_rule == "HMSTDP":
            s = s.flatten()

        s_shape = [1] * len(s.shape[1:])
        print(s)
        # Run the agent for time t on state s with reward r
        inputs = {k: s.repeat(agent.sqn.time, *s_shape) for k in agent.sqn.inputs}
        print(inputs)
        agent.sqn.run(inputs=inputs, reward=r)

        # Update output spikes
        if agent.sqn.output is not None:
            agent.sqn.spike_record["Output"] = (
                agent.sqn.network.monitors["output_monitor"].get("s").float()
            )

        total_reward += r

        global s_im
        global s_ax
        s_im, s_ax = plot_spikes({
            "input": agent.sqn.network.monitors["input_monitor"].get("s"),
            "hidden": agent.sqn.network.monitors["hidden_monitor"].get("s"),
            "output": agent.sqn.network.monitors["output_monitor"].get("s")
        }, ims=s_im, axes=s_ax)

    return total_reward

In [8]:
def get_env_dim(env: gym.Env) -> Tuple[int, int]:
    """Returns input_dim & output_dim
    Args:
        env (gym.Env): gym Environment
    Returns:
        int: input_dim
        int: output_dim
    """
    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n

    return input_dim, output_dim

In [9]:
def clip_reward(reward):
    """Clip reward so that it's in [-1, 1]
    """
    if reward < -1:
        reward = -1
    elif reward > 1:
        reward = 1
    return reward

In [12]:
try:
    env = GymEnvironment(FLAGS.env)

    average_rewards = []
    q = deque(maxlen=100)

    input_dim, output_dim = get_env_dim(env.env)
    agent = Agent(80 * 80, [1, 1, 80, 80], output_dim, FLAGS.hidden_dim)

    for i in range(FLAGS.n_episode):
        r = play_episode(env, agent)
        print("[Episode: {:5}] Reward: {:5}".format(i + 1, r))

        q.append(r)
        if i % 100 == 0:
            average_rewards.append(mean(q))

    name = "SQN-{}-{}-{}-{}-reward_clamping".format(FLAGS.update_rule.replace(" ", ""), FLAGS.env, FLAGS.n_episode, FLAGS.gamma)

    fig, ax = plt.subplots()
    ax.plot(average_rewards)

    ax.set(xlabel='Episode', ylabel='Reward',
           title='SQN performance with {} on {}'.format(FLAGS.update_rule, FLAGS.env))
    plt.show()


finally:
    env.close()

tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  

tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  

tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  

{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
tensor([1., 1., 1.,  ..., 1., 1., 1.])
{'Input': tensor([1.,

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>